# PurpleAir Stations QAQC

## Import Packages

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

## Definitions

In [2]:
# Getting .env information (PurpleAir API Read Key)
from dotenv import load_dotenv

load_dotenv()

api = os.getenv('PURPLEAIR_API_TOKEN')

In [3]:
#Setting lat/long for PurpleAir API Parameters

datapath = os.path.join('..', '..', 'Data')

extent = gpd.read_file(os.path.join(datapath, 'extent.geojson'))

nwlng, selat, selng, nwlat = extent.to_crs('EPSG:4326').total_bounds # Change coordinate reference system

## Importing PurpleAir Station Data from PurpleAir API

In [4]:
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

In [5]:
#Set bounding strings for API parameters
bounds_strings = [f'nwlng={nwlng}',
                  f'nwlat={nwlat}',
                  f'selng={selng}',
                  f'selat={selat}']

bounds_string = '&'.join(bounds_strings)

print(bounds_string)

nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [6]:
#Setting parameters for API
fields = ['firmware_version','date_created','last_modified','last_seen', 'name', 'uptime','position_rating','channel_state','channel_flags','altitude',
          'location_type','latitude', 'longitude']

fields_string = 'fields=' + '%2C'.join(fields)

print(fields_string)

fields=firmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cname%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude


In [7]:
#finalizing query for API function
query_string = '&'.join([fields_string, bounds_string])

print(query_string)

fields=firmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cname%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude&nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [8]:
#calling the API
response = getSensorsData(query_string, api)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=firmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cname%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude&nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [9]:
response.json().keys()

dict_keys(['api_version', 'time_stamp', 'data_time_stamp', 'max_age', 'firmware_default_version', 'fields', 'location_types', 'channel_states', 'channel_flags', 'data'])

In [10]:
response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

df = pd.DataFrame(data, columns = col_names)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sensor_index      95 non-null     object
 1   last_modified     95 non-null     object
 2   date_created      95 non-null     object
 3   last_seen         95 non-null     object
 4   name              95 non-null     object
 5   location_type     95 non-null     object
 6   firmware_version  95 non-null     object
 7   uptime            95 non-null     object
 8   position_rating   95 non-null     object
 9   latitude          95 non-null     object
 10  longitude         95 non-null     object
 11  altitude          95 non-null     object
 12  channel_state     95 non-null     object
 13  channel_flags     95 non-null     object
dtypes: object(14)
memory usage: 10.5+ KB


In [11]:
df.head(1)

,sensor_index,last_modified,date_created,last_seen,name,location_type,firmware_version,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags
0,3088,1504993349,1504040633,1698938031,Howe Neighborhood,0,6.06b,22445,5,44.935818,-93.21752,833,3,1


### Compare with the list of Ids from City

In [12]:
# Load the list the City provided (or any other list

# sensor_info = pd.read_excel(os.path.join(datapath, 'PA IDs and indexes.xlsx')) # Load as DataFrame

# ids_from_city = sensor_info['Sensor Index'].dropna().astype(int) # List from the City

sensor_info = gpd.read_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

ids_from_previous_run = sensor_info.sensor_index.unique().astype(int)

In [13]:
len(ids_from_previous_run)

66

In [14]:
# From what we just queried from PurpleAir 
# Only want the City's monitors

is_city = df.name.apply(lambda x: 'CITY OF MINNEAPOLIS' in x.upper())

ids_from_PurpleAir = df[is_city].sensor_index.astype(int)

In [15]:
len(ids_from_PurpleAir)

49

In [16]:
# Now let's get the union of these two lists of ids

ids_from_either = np.union1d(ids_from_previous_run, ids_from_PurpleAir)

In [17]:
len(ids_from_either)

66

### Final PurpleAir Query

In [18]:
# One final query for this list of ids

# Get start Times

sensor_string = 'show_only=' + '%2C'.join(ids_from_either.astype(str))

query_string = '&'.join([fields_string+'%2Cpm2.5_10minute', sensor_string])

In [19]:
response = getSensorsData(query_string, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=firmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cname%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude%2Cpm2.5_10minute&show_only=142718%2C142720%2C142724%2C142726%2C142728%2C142730%2C142732%2C142734%2C142736%2C142738%2C142744%2C142748%2C142750%2C142752%2C142756%2C142772%2C142774%2C142926%2C143214%2C143216%2C143222%2C143224%2C143226%2C143238%2C143240%2C143242%2C143246%2C143248%2C143634%2C143636%2C143648%2C143656%2C143660%2C143666%2C143668%2C143916%2C143942%2C143944%2C145202%2C145204%2C145242%2C145250%2C145454%2C145470%2C145498%2C145502%2C145504%2C145506%2C145604%2C145610%2C145614%2C145616%2C156605%2C157747%2C157757%2C157785%2C157837%2C157845%2C157861%2C157871%2C157877%2C157935%2C166459%2C168327%2C177765%2C194635


## Cleaning PurpleAir Station Data

In [20]:
#find outside sensors
df_outside = sensors_df[sensors_df.location_type == 0]
len(df_outside)

66

In [21]:
#drop the location_type now that we have filtered for outdoor sensors only
df_stations = df_outside.drop('location_type', axis=1)

In [22]:
#Converting UNIX date/time to pd 

# THESE ARE IN UTC time!!! - Need to correct this in the workflow!

df_stations['last_modified'] = pd.to_datetime(df_stations['last_modified'], unit='s')
df_stations['date_created'] = pd.to_datetime(df_stations['date_created'], unit='s')
df_stations['last_seen'] = pd.to_datetime(df_stations['last_seen'], unit='s')

In [23]:
df_stations.head(3)

,sensor_index,last_modified,date_created,last_seen,name,firmware_version,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags,pm2.5_10minute
0,142718,2023-08-02 18:50:17,2022-01-12 18:57:49,2023-11-02 15:13:50,City of Minneapolis Community Air Monitoring P...,7.02,8219,5,44.99631,-93.29565,857,3,0,19.2
1,142720,2023-08-02 18:06:36,2022-01-12 18:57:55,2023-11-02 15:14:09,City of Minneapolis Community Air Monitoring P...,7.02,28451,5,44.955555,-93.254974,850,3,0,20.2
2,142726,2023-02-02 17:31:06,2022-01-12 18:58:17,2023-11-02 15:13:33,City of Minneapolis Community Air Monitoring P...,7.02,4403,5,45.01507,-93.28903,889,3,0,24.6


In [25]:
dt.datetime.now()

datetime.datetime(2023, 11, 2, 10, 17, 28, 622332)

In [24]:
# Sensor Flags

# Key
# Channel State -  0 = No PM, 3 = Both On
# Channel Flags - 0 = Normal, 1 = A Downgraded, 2 - B Downgraded, 3 - Both Downgraded

pd.set_option('display.max_colwidth', None)

df = df_stations.rename(columns = {'pm2.5_10minute':'pm25'})


flagged = (df.channel_flags != 0) | (df.channel_state == 1) | (df.last_seen < dt.datetime.now())

flagged_sensors = df[flagged][['sensor_index', 
                               'channel_flags', 
                               'channel_state',
                                'last_seen',
                               'name', 'pm25']]

not_flagged_sensors = df[~flagged][['sensor_index', 
                               'channel_flags',
                               'channel_state',
                                'last_seen',
                               'name', 'pm25']]

print(len(flagged_sensors))

flagged_sensors.sort_values('last_seen').reset_index(drop=True)#.to_csv('flagged_sensors.csv')

27


,sensor_index,channel_flags,channel_state,last_seen,name,pm25
0,142926,0,3,2022-11-10 16:26:39,City of Minneapolis Community Air Monitoring Project 22,9.6
1,145504,0,3,2023-01-09 22:09:39,City of Minneapolis Community Air Monitoring Project 4,25.6
2,142772,0,3,2023-03-28 19:34:07,City of Minneapolis community Air Monitoring Project 31,4.4
3,177765,3,0,2023-03-28 19:37:14,City of Minneapolis Community Air Monitoring Project 28,None
4,143238,0,3,2023-04-26 20:33:55,City of Minneapolis Community Air Monitoring Project 35,5.8
5,142752,0,3,2023-06-22 15:19:40,City of Minneapolis Community Air Monitoring Project 40,27.0
6,143222,0,3,2023-06-25 04:07:54,City of Minneapolis,6.4
7,143944,0,3,2023-06-27 12:01:58,City of Minneapolis Community Air Monitoring Project 37,79.0
8,145202,0,3,2023-06-29 19:20:54,City of Minneapolis Community Air Monitoring Project 60,28.3
9,145610,0,3,2023-07-03 11:22:30,City of Minneapolis Community Air Monitoring Project 24,32.8


In [119]:
# Save as CSV

df_stations.to_csv(os.path.join(datapath, 'PurpleAir_Stations.csv'))

# Save as geojson

gdf_stations = gpd.GeoDataFrame(df_stations, 
                                geometry = gpd.points_from_xy(
                                    df_stations.longitude,
                                    df_stations.latitude,
                                    crs = 'EPSG:4326')
                               ).to_crs('EPSG:26915')

cols_for_db = ['sensor_index'] + fields[:-3] + ['geometry']

sorted_gdf = gdf_stations[cols_for_db]

sorted_gdf.to_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

## Insert Data into SQL Table - Not Quite Done

This doesn't check to see if the sensors are already in the database

In [33]:
cols_for_db

['sensor_index',
 'firmware_version',
 'date_created',
 'last_modified',
 'last_seen',
 'name',
 'uptime',
 'position_rating',
 'channel_state',
 'channel_flags',
 'altitude',
 'geometry']

In [34]:
# Sort the df and correct values

sorted_df = sorted_gdf.drop('geometry', axis=1).copy()

sorted_df['wkt'] = sorted_gdf.geometry.apply(lambda x: x.wkt)

sorted_df['date_created'] = sorted_gdf.date_created.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))
sorted_df['last_modified'] = sorted_gdf.last_modified.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))
sorted_df['last_seen'] = sorted_gdf.last_seen.apply(lambda x : x.strftime('%Y-%m-%d %H:%M:%S'))

# Replace non-numeric nans

# sorted_df['firmware_version'] = sorted_df['firmware_version'].fillna(value = 'a', inplace = True)
# sorted_df['firmware_version'] = sorted_df['firmware_version'].fillna(value = 'NaN', inplace = True)

In [35]:
sorted_df.head()

,sensor_index,firmware_version,date_created,last_modified,last_seen,name,uptime,position_rating,channel_state,channel_flags,altitude,wkt
0,142718,7.02,2022-01-12 18:57:49,2023-08-02 18:50:17,2023-10-17 22:30:38,City of Minneapolis Community Air Monitoring P...,69679,5,3,0,857,POINT (476696.84844637447 4982582.8198003275)
1,142720,7.02,2022-01-12 18:57:55,2023-08-02 18:06:36,2023-10-17 22:31:05,City of Minneapolis Community Air Monitoring P...,5848,5,3,0,850,POINT (479888.6697801179 4978044.582460884)
2,142726,7.02,2022-01-12 18:58:17,2023-02-02 17:31:06,2023-10-04 14:56:05,City of Minneapolis Community Air Monitoring P...,2,5,3,0,889,POINT (477226.07726585865 4984664.938670933)
3,142724,7.02,2022-01-12 18:58:09,2023-08-02 16:12:05,2023-10-17 22:29:51,City of Minneapolis Community Air Monitoring P...,1690,5,3,0,859,POINT (480783.00497330126 4976062.035271977)
4,142730,7.02,2022-01-12 18:58:36,2023-08-02 15:41:14,2023-09-25 14:18:30,City of Minneapolis Community Air Monitoring P...,816,5,3,0,858,POINT (476695.09256082535 4982102.367948439)


In [28]:
## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds)) 

In [31]:
# Connect to PostGIS Database

conn = psycopg2.connect(**pg_connection_dict)
cur = conn.cursor()


# iterate over the dataframe and insert each row into the database using a SQL INSERT statement

for index, row in sorted_df.copy().iterrows():

    q1 = sql.SQL('INSERT INTO "PurpleAir Stations" ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_Transform(ST_SetSRID(ST_GeomFromText(%s), 26915),4326)::geometry'))
    # print(q1.as_string(conn))
    # print(row)
    # break
    
    cur.execute(q1.as_string(conn),
        (list(row.values))
        )
    # Commit command

    conn.commit()

# Close the cursor and connection
cur.close()
conn.close()